In [1]:
"""Reads data csv data from path and stores partitioned data to a local filepath as a csv.
Usage: split_data.py --path=<path> --out_file=<out_file> 
 
Options:
--path=<path>               Path to read file from 
--score_file=<score_file>   Path (including filename) of where to locally save cross val score
--model_path=<model_path>   Path for the model pickle file
"""

'Reads data csv data from path and stores partitioned data to a local filepath as a csv.\nUsage: split_data.py --path=<path> --out_file=<out_file> \n \nOptions:\n--path=<path>               Path to read file from \n--score_file=<score_file>   Path (including filename) of where to locally save cross val score\n--model_path=<model_path>   Path for the model pickle file\n'

In [2]:
import sys
import time
import os
import string

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import HTML
import mglearn
from IPython.display import display

from sklearn.dummy import DummyClassifier, DummyRegressor

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    PolynomialFeatures,
    StandardScaler,
)


from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import RFE, RFECV
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, Ridge, RidgeCV
from sklearn.metrics import make_scorer
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    ShuffleSplit,
    cross_val_score,
    cross_validate,
    train_test_split,
)

from sklearn.svm import SVC, SVR

%matplotlib inline

In [12]:
# opt = docopt(__doc__)


def main(path, out_file, model_path):
    train_df = pd.read_csv(path)
    
    X_train, y_train = train_df.drop(columns=["Contraceptive_method_used"]), train_df["Contraceptive_method_used"]
#     X_test, y_test = test_df.drop(columns=["Contraceptive_method_used"]), test_df["Contraceptive_method_used"]

    #Converting it to a binary model(train set)
    y_train = y_train.replace(1,0)
    y_train = y_train.replace([2,3],1)

    numeric_features = ['Wife_age', 'Number_of_children_ever_born']
    ordinal_features = ["Wife_education","Husband_education","Husband_occupation", "Standard_of_living_index"]
    passthrough_features = ['Wife_religion','Wife_now_working?','Media_exposure'] 


    preprocessor = make_column_transformer(
        (make_pipeline(SimpleImputer(), StandardScaler()),numeric_features,),
        (OrdinalEncoder(),ordinal_features,),  
        ("passthrough", passthrough_features),)
    
    models_bal = {
    "decision tree": DecisionTreeClassifier(random_state=123),
    "kNN": KNeighborsClassifier(),
    "Logistic Regression": LogisticRegression(max_iter=100, multi_class='ovr',random_state=123 ),
    "RBF SVM": SVC(random_state=123),}
    
    results_bal = {}
    results_bal_f = {}

    for keys, value in models_bal.items():
        pipe_bal = make_pipeline(preprocessor, value)
        results_bal[keys] = cross_validate(pipe_bal, X_train, y_train, cv = 5, return_train_score = True)
        results_bal_f[keys] = pd.DataFrame(results_bal[keys]).mean()
    
    try:
        pd.DataFrame(results_bal_f).to_csv(out_file)
    except:
        os.makedirs(os.path.dirname(out_file))
        pd.DataFrame(results_bal_f).to_csv(out_file)

# if __name__ == "__main__":
#     main(opt["--path"], opt["--score_file"],opt["--model_path"])

In [13]:
main('../data/processed/train.csv','../results/val_score_results.csv', None)